In [1]:
import os
from datetime import datetime as dt
from matplotlib import pyplot as plt
import numpy as np
import xarray as xr

import sys
sys.path.append('/Users/danstechman/GoogleDrive/School/Research/PECAN/Radar/code/samuraiAnalysis/')
from getFLpathData import getFLpathData
from getVarLims import getVarLims
from plotFLpath import plotFLpath
from samPlt import initMapPlot, initPlot, plotContour, plotVec
from samImport import samImport


%matplotlib inline


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119 


In [3]:
### Which case do you want to run? ###
# This is the name of the directory where the SAMURAI
# output is located
outPrefix = 'P6_D7_fsdabr'
# outPrefix = 'P6_fsd'
# outPrefix = 'P6_fsdabr'
# outPrefix = 'P6_fsdMultV'
# outPrefix = 'S3_fsd'
# outPrefix = 'S3_fsdabr'
# outPrefix = 'S3_fsdMultV'
# outPrefix = 'S3P6_fsdabr'
# outPrefix = 'S3prl1'
# outPrefix = 'S3prl1_fsdabr'


### Set various file path variables ###
samPrefix = '/Users/danstechman/GoogleDrive/PECAN-Data/samurai/20150706/'
samFile = samPrefix + outPrefix + '/output/samurai_XYZ_analysis.nc'
savePath = samPrefix + outPrefix + '/figs/'
flFile = '/Users/danstechman/GoogleDrive/PECAN-Data/FlightLevelData/20150706I1_AC.nc'

### Import SAMURAI data ###
samData = samImport(samFile)

gribFile = '/Users/danstechman/GoogleDrive/PECAN-Data/model/RAP/20150706/rap_130_20150706_0400_000.nc'
gribData = xr.open_dataset(gribFile)

### Which variables do we want to plot, at what levels (levels are in km), ###
###     cartesian or map coordinates ('xy' or 'map'), and do you want      ###
###     to save the figures?                                               ###
pltDBZ  = True
pltU    = False
pltV    = False
pltW    = False
pltVort = False

savePlt = True

crds = 'map'

pltLevs = samData['alt']
# pltLevs = [1]

### Plot flight track on analysis? ###
# flS and flE indicate the start and end date/time for the track.
# For the data index of pddSprlTimes, use (leg/sprl # - 1).
pltFT   = True
pddSprlTimes = np.load('/Users/danstechman/GoogleDrive/School/Research/PECAN/Radar/code/samuraiAnalysis/pecan_PDD-Sprl_times/20150706_pddSprl_times.npy')[()]
flS = pddSprlTimes['I20P_strtDT'][5]
flE = pddSprlTimes['I20P_endDT'][5]
# flS = pddSprlTimes['I20S_strtDT'][2]
# flE = pddSprlTimes['I20S_endDT'][2]

#Purl 1 from Spiral 3:
# flS = dt.strptime('20150706-042710','%Y%m%d-%H%M%S')
# flE = dt.strptime('20150706-043152','%Y%m%d-%H%M%S')

##########################################
### End of commonly modified variables ###
##########################################

runId = outPrefix

dT = samData['time']

flLat,flLon = getFLpathData(flFile,pathStrt=flS,pathEnd=flE,crdsOnly=True)


if not os.path.exists(savePath):
    os.makedirs(savePath)

if dT is not None:
    dtSave = dt.strftime(dT,'%Y%m%d_%H%M')
else:
    dtSave = 'unknownDT'

if savePlt:
    if runId:
        runIdSv = '_' + runId.replace('_','-')

    

ix = 1
for lev in pltLevs:
    
    print('Now plotting {:.2f} km ({} of {})...'.format(lev,ix,len(pltLevs)))

    if pltDBZ:
        fig,ax,grd,proj = plotContour(samData['dbz'],lev,samData['lon'],samData['lat'],
                                      crds,'dbz',figsize=(10,10),dT=dT,vLimMthd='tight',
                                      runId=runId)
        
        if pltFT and crds is 'map':
            plotFLpath(proj,flLon,flLat,dubLine=True)
            
        u = gribData.UGRD_P0_L100_GLC0.to_masked_array().squeeze()
        v = gribData.VGRD_P0_L100_GLC0.to_masked_array().squeeze()
        gLat = gribData.gridlat_0.to_masked_array().squeeze()
        gLon = gribData.gridlon_0.to_masked_array().squeeze()
        
        plotVec(samData['lon'][1::5],samData['lat'][1::5],samData['u'][lev*2,1::5,1::5],
               samData['v'][lev*2,1::5,1::5],crds,proj=proj)
        
#         plotVec(gLon,gLat,u[32,:,:],v[32,:,:],crds,proj=proj,color='white',linewidth=1.5,
#                 edgecolor='white',quivKeyX=0.08)
#         plotVec(gLon,gLat,u[32,:,:],v[32,:,:],crds,proj=proj,color='blue',edgecolor=None,
#                 quivKeyX=0.08)
        
        
        if savePlt:
            saveStr = '{}{}{}_{}_{}_{}km.png'.format(savePath,dtSave,runIdSv,crds,'dbz',lev)
            fig.savefig(saveStr,bbox_inches='tight')
            plt.close()
            
    if pltU:
        fig,ax,grd,proj = contourSamurai(samData['u'],lev,samData['lon'],samData['lat'],crds,'u',figsize=(10,10),pltVec=True,
                              quivU=samData['u'],quivV=samData['v'],quivKeySpd=20,quivKeyUnits='m/s',dT=dT,
                              vLimMthd='tightM',noDisp=True,runId=runId)
        
        if pltFT and crds is 'map':
            plotFLpath(proj,flLon,flLat,dubLine=True)
        
        if savePlt:
            saveStr = '{}{}{}_{}_{}_{}km.png'.format(savePath,dtSave,runIdSv,crds,'u',lev)
            fig.savefig(saveStr,bbox_inches='tight')
            plt.close()
    
    if pltV:
        fig,ax,grd,proj = contourSamurai(samData['v'],lev,samData['lon'],samData['lat'],crds,'v',figsize=(10,10),pltVec=True,
                              quivU=samData['u'],quivV=samData['v'],quivKeySpd=20,quivKeyUnits='m/s',dT=dT,
                              vLimMthd='tightM',noDisp=True,runId=runId)
        
        if pltFT and crds is 'map':
            plotFLpath(proj,flLon,flLat,dubLine=True)
        
        if savePlt:
            saveStr = '{}{}{}_{}_{}_{}km.png'.format(savePath,dtSave,runIdSv,crds,'v',lev)
            fig.savefig(saveStr,bbox_inches='tight')
            plt.close()
    
    if pltW:
        fig,ax,grd,proj = contourSamurai(samData['w'],lev,samData['lon'],samData['lat'],crds,'w',figsize=(10,10),pltVec=True,
                              quivU=samData['u'],quivV=samData['v'],quivKeySpd=20,quivKeyUnits='m/s',dT=dT,
                              vLimMthd='tightM',noDisp=True,runId=runId)
        
        if pltFT and crds is 'map':
            plotFLpath(proj,flLon,flLat,dubLine=True)
        
        if savePlt:
            saveStr = '{}{}{}_{}_{}_{}km.png'.format(savePath,dtSave,runIdSv,crds,'w',lev)
            fig.savefig(saveStr,bbox_inches='tight')
            plt.close()
    
    if pltVort:
        fig,ax,grd,proj = contourSamurai(samData['vort'],lev,samData['lon'],samData['lat'],crds,'u',figsize=(10,10),pltVec=True,
                              quivU=samData['u'],quivV=samData['v'],quivKeySpd=20,quivKeyUnits='m/s',dT=dT,
                              vLimMthd='tightEM',noDisp=True,runId=runId)
        
        if pltFT and crds is 'map':
            plotFLpath(proj,flLon,flLat,dubLine=True)
        
        if savePlt:
            saveStr = '{}{}{}_{}_{}_{}km.png'.format(savePath,dtSave,runIdSv,crds,'vort',lev)
            fig.savefig(saveStr,bbox_inches='tight')
            plt.close()
    
    ix += 1

Now plotting 0.00 km (1 of 33)...


/Users/danstechman/anaconda/lib/python3.5/site-packages/numpy/ma/core.py:3158: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  dout = self.data[indx]
/Users/danstechman/anaconda/lib/python3.5/site-packages/numpy/ma/core.py:3213: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  dout._mask = _mask[indx]


Now plotting 0.50 km (2 of 33)...
Now plotting 1.00 km (3 of 33)...
Now plotting 1.50 km (4 of 33)...
Now plotting 2.00 km (5 of 33)...
Now plotting 2.50 km (6 of 33)...
Now plotting 3.00 km (7 of 33)...
Now plotting 3.50 km (8 of 33)...
Now plotting 4.00 km (9 of 33)...
Now plotting 4.50 km (10 of 33)...
Now plotting 5.00 km (11 of 33)...
Now plotting 5.50 km (12 of 33)...
Now plotting 6.00 km (13 of 33)...
Now plotting 6.50 km (14 of 33)...
Now plotting 7.00 km (15 of 33)...
Now plotting 7.50 km (16 of 33)...
Now plotting 8.00 km (17 of 33)...
Now plotting 8.50 km (18 of 33)...
Now plotting 9.00 km (19 of 33)...
Now plotting 9.50 km (20 of 33)...
Now plotting 10.00 km (21 of 33)...
Now plotting 10.50 km (22 of 33)...
Now plotting 11.00 km (23 of 33)...
Now plotting 11.50 km (24 of 33)...
Now plotting 12.00 km (25 of 33)...
Now plotting 12.50 km (26 of 33)...
Now plotting 13.00 km (27 of 33)...
Now plotting 13.50 km (28 of 33)...
Now plotting 14.00 km (29 of 33)...
Now plotting 14.50 